<a href="https://colab.research.google.com/github/acmoral/SistemasInteligentes/blob/main/taller1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Taller 1
### [Introducción a los sistemas inteligentes 2020-1](https://fagonzalezo.github.io/iis-2020-1/)

Integrantes del grupo (máximo 3):

* Brayan Fabian Prieto Cortes - bfprietoc
* Sebastian Aguilera Novoa - saguileran
* Andrea Carolina Mora López - acmoral

___________

**Fecha límite de entrega**: _Lunes 13 de Abril_ antes de la medianoche 
Envios fuera de este plazo no se tendrán en cuenta.


Cerciórese de reiniciar y correr el notebook en su totalidad antes de enviarlo. Verifique que todas las salidas se muestran de manera correcta.

**Instrucciones de envío:**

Este notebook debe enviarse a través del siguiente [File Request](https://www.dropbox.com/request/OapI3DaGbxdtWoUkpLiR) antes de la hora límite. El archivo debe nombrarse como  `isi-taller1-unalusername1-unalusername2-unalusername3.ipynb`, donde unalusername es el nombre de usuario asignado por la universidad (incluya los nombres de usuario de todos los miembros del grupo).

___________

## Cuadrado de Rubik (o Cubo de Rubik 2D)

El *Cuadrado de Rubik* es un rompecabezas inspirado en el famoso cubo de Rubik. Consiste de un arreglo de 16 fichas organizadas en una matriz de $4\times 4$ como se ilustra en la siguiente figura:

<img src="https://raw.githubusercontent.com/fagonzalezo/iis-2018-2/master/rubik2D.png" 
alt="Cuadrado de Rubik " width="240" height="180" border="10" />

Los colores son ilustrativos, lo importante es el número en cada una de las fichas. Se pueden hacer 10 movimientos diferentes correspondientes a rotar las 4 fichas alrededor de cada uno de los puntos A, B, C, D y E en el sentido de las manecillas del reloj o en el sentido opuesto.

Su objetivo es modelar el *Cuadrado de Rubik* como un problema de búsqueda y resolverlo usando diferentes algoritmos de búsqueda.

_________


####Códigos de AIMA

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
import time
from time import clock

from collections import defaultdict, deque, Counter
from itertools import combinations

class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        
        
def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]


#-------------------------------------------------------
FIFOQueue = deque

LIFOQueue = list

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)


#-------------------------------------------------------
def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    reached = {problem.initial: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def best_first_tree_search(problem, f):
    "A version of best_first_search without the `reached` table."
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            if not is_cycle(child):
                frontier.add(child)
    return failure


def g(n): return n.path_cost


def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + h(n))


def astar_tree_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n), with no `reached` table."""
    h = h or problem.h
    return best_first_tree_search(problem, f=lambda n: g(n) + h(n))


def weighted_astar_search(problem, h=None, weight=1.4):
    """Search nodes with minimum f(n) = g(n) + weight * h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=lambda n: g(n) + weight * h(n))

        
def greedy_bfs(problem, h=None):
    """Search nodes with minimum h(n)."""
    h = h or problem.h
    return best_first_search(problem, f=h)


def uniform_cost_search(problem):
    "Search nodes with minimum path cost first."
    return best_first_search(problem, f=g)


def breadth_first_bfs(problem):
    "Search shallowest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=len)


def depth_first_bfs(problem):
    "Search deepest nodes in the search tree first; using best-first."
    return best_first_search(problem, f=lambda n: -len(n))


def is_cycle(node, k=30):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):
        return (ancestor is not None and k > 0 and
                (ancestor.state == node.state or find_cycle(ancestor.parent, k - 1)))
    return find_cycle(node.parent, k)

def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result
        
def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result

def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    reached = {problem.initial}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure


#--------------------------------------------------------------------------------
class CountCalls:
    """Delegate all attribute gets to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()
        
    def __getattr__(self, attr):
        "Delegate to the original object, after incrementing a counter."
        self._counts[attr] += 1
        #print(attr)
        return getattr(self._object, attr)

        
def report(searchers, problems, verbose=True):
    """Show summary statistics for each searcher (and on each problem unless verbose is false)."""
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob   = CountCalls(p)
            soln   = searcher(prob)
            counts = prob._counts; 
            counts.update(actions=len(soln), cost=soln.path_cost)
            total_counts += counts
            #print(prob, soln, counts, total_counts)
            if verbose: report_counts(counts, str(p)[:40])
        report_counts(total_counts, 'TOTAL\n')
        
def report_counts(counts, name):
    """Print one line of the counts report."""
    print('{:9,d} nodes |{:9,d} goal |{:5.0f} cost |{:8,d} actions | {}'.format(
          counts['result'], counts['is_goal'], counts['cost'], counts['actions'], name))           
    

#------------------------------------
def bidirectional_best_first_search(problem_f, f_f, problem_b, f_b, terminated):
    node_f = Node(problem_f.initial)
    node_b = Node(problem_f.goal)
    frontier_f, reached_f = PriorityQueue([node_f], key=f_f), {node_f.state: node_f}
    frontier_b, reached_b = PriorityQueue([node_b], key=f_b), {node_b.state: node_b}
    solution = failure
    while frontier_f and frontier_b and not terminated(solution, frontier_f, frontier_b):
        def S1(node, f):
            return str(int(f(node))) + ' ' + str(path_states(node))
        print('Bi:', S1(frontier_f.top(), f_f), S1(frontier_b.top(), f_b))
        if f_f(frontier_f.top()) < f_b(frontier_b.top()):
            solution = proceed('f', problem_f, frontier_f, reached_f, reached_b, solution)
        else:
            solution = proceed('b', problem_b, frontier_b, reached_b, reached_f, solution)
    return solution

def inverse_problem(problem):
    if isinstance(problem, CountCalls):
        return CountCalls(inverse_problem(problem._object))
    else:
        inv = copy.copy(problem)
        inv.initial, inv.goal = inv.goal, inv.initial
        return inv

def bidirectional_uniform_cost_search(problem_f):
    def terminated(solution, frontier_f, frontier_b):
        n_f, n_b = frontier_f.top(), frontier_b.top()
        return g(n_f) + g(n_b) > g(solution)
    return bidirectional_best_first_search(problem_f, g, inverse_problem(problem_f), g, terminated)

def bidirectional_astar_search(problem_f):
    def terminated(solution, frontier_f, frontier_b):
        nf, nb = frontier_f.top(), frontier_b.top()
        return g(nf) + g(nb) > g(solution)
    problem_f = inverse_problem(problem_f)
    return bidirectional_best_first_search(problem_f, lambda n: g(n) + problem_f.h(n),
                                           problem_b, lambda n: g(n) + problem_b.h(n), 
                                           terminated)
   

def proceed(direction, problem, frontier, reached, reached2, solution):
    node = frontier.pop()
    for child in expand(problem, node):
        s = child.state
        print('proceed', direction, S(chhamming_distanceild))
        if s not in reached or child.path_cost < reached[s].path_cost:
            frontier.add(child)
            reached[s] = child
            if s in reached2: # Frontiers collide; solution found
                solution2 = (join_nodes(child, reached2[s]) if direction == 'f' else
                             join_nodes(reached2[s], child))
                #print('solution', path_states(solution2), solution2.path_cost, 
                # path_states(child), path_states(reached2[s]))
                if solution2.path_cost < solution.path_cost:
                    solution = solution2
    return solution

S = path_states

#A-S-R + B-P-R => A-S-R-P + B-P
def join_nodes(nf, nb):
    """Join the reverse of the backward node nb to the forward node nf."""
    #print('join', S(nf), S(nb))
    join = nf
    while nb.parent is not None:
        cost = join.path_cost + nb.path_cost - nb.parent.path_cost
        join = Node(nb.parent.state, join, nb.action, cost)
        nb = nb.parent
        #print('  now join', S(join), 'with nb', S(nb), 'parent', S(nb.parent))
    return join

    
def hamming_distance(A, B):
    "Number of positions where vectors A and B are different."
    return sum(a != b for a, b in zip(A, B))

#### Clase Rubik2D_problem

In [ ]:
# This is the Problem class from AIMA, you don't have to modify it

class Problem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other 
    keyword args for the subclass."""

    def __init__(self, initial=None, goal=None, **kwds): 
        self.__dict__.update(initial=initial, goal=goal, **kwds) 
        
    def actions(self, state):        raise NotImplementedError
    def result(self, state, action): raise NotImplementedError
    def is_goal(self, state):        return state == self.goal
    def action_cost(self, s, a, s1): return 1
    def h(self, node):               return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

class rubik2d_problem(Problem):    
    
    def __init__(self, initial=None, goal=(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16), **kwds): 
        self.__dict__.update(initial=tuple(initial), goal=goal, **kwds) 
    
    def actions(self, state):
        """Return a list of actions that can be executed in the given state."""
        return  ["A+"], ["A-"], ["B+"], ["B-"], ["C+"], ["C-"], ["D+"], ["D-"], ["E+"], ["E-"]
        #["A+"],["B+"], ["C+"], ["D+"], ["E+"], ["F+"], ["A-"], ["B-"], ["C-"], ["E-"]

    def result(self, state, action):
        """Return the state that results from executing the given
        action at the given state. The action must be one of
        self.actions(state)."""
        result, resultp = list(state), list(state)
        act, *_ = action

        def ClockwiseTurn(i):
          resultp[i]   =  result[i-1]
          resultp[i-1] =  result[i+3]
          resultp[i+3] =  result[i+4]
          resultp[i+4] =  result[i]

        def AntiClockwiseTurn(i):
          resultp[i]   =  result[i+4]
          resultp[i-1] =  result[i]
          resultp[i+3] =  result[i-1]
          resultp[i+4] =  result[i+3]

        if   act == "A+": ClockwiseTurn(1)
        elif act == "A-": AntiClockwiseTurn(1)
        elif act == "B+": ClockwiseTurn(3)
        elif act == "B-": AntiClockwiseTurn(3)
        elif act == "C+": ClockwiseTurn(6)
        elif act == "C-": AntiClockwiseTurn(6)
        elif act == "D+": ClockwiseTurn(9)
        elif act == "D-": AntiClockwiseTurn(9)
        elif act == "E+": ClockwiseTurn(11)
        elif act == "E-": AntiClockwiseTurn(11)
        
        return tuple(resultp)
     
    def h(self, node):
        """The Manhattan heuristic."""
        X = (0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3)
        Y = (0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3)
        return sum(abs(X[s-1] - X[g-1]) + abs(Y[s-1] - Y[g-1]) for (s, g) in zip(node.state, self.goal) if s != 0)

    def h0(self, node):
        """The null heuristic."""
        return 0

    def h1(self, node):
        """The misplaced tiles heuristic."""
        return hamming_distance(node.state, self.goal)
    
    def h2(self, node):
        """The Neighborhood heuristic."""
        count = 0
        for i in range(len(node.state)-1):
          if node.state[i+1] - node.state[i] != 1: count += 1
        return count   
    
    def h3(self, node):
        """The pairs heuristic."""
        count = 0
        for i in range(0,len(node.state)-1,2):
          if node.state[i+1] - node.state[i] != 1: count += 1
        return count    
      
    def Print(self): print(" ", *self.initial[:4], "\n ", *self.initial[4:8], "\n ",
                              *self.initial[8:12], "\n ", *self.initial[12:], "\n") 

### 2. Evalue su código con diferentes estrategias de búsqueda



Estados iniciales sugeridos con sus correspondientes problemas y resultados

In [ ]:
import random
#Random state
def Random():
    random.shuffle(list(range(16)))
    return tuple(x)
#Estados iniciales a evaluar
p0 = rubik2d_problem([3, 2, 7, 4, 1, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16]) #A+, B+
p1 = rubik2d_problem([1, 2, 7, 3, 5, 9, 6, 4, 13, 11, 12, 16, 14, 10, 8, 15])
p2 = rubik2d_problem([2, 7, 4, 8, 1, 5, 3, 11, 14, 13, 15, 10, 6, 9, 16, 12])
p3 = rubik2d_problem([1, 9, 4, 8, 5, 6, 3, 2, 15, 10, 11, 12, 13, 14, 7, 16])

AllFunctions = [uniform_cost_search,  breadth_first_search, breadth_first_bfs, iterative_deepening_search, astar_search,
              greedy_bfs, weighted_astar_search] #, extra_weighted_astar_search, depth_limited_search]
p, functions = [p1, p2, p3], [breadth_first_search, uniform_cost_search, iterative_deepening_search]
%time results = [(str(fun)[10:-19]+" - p"+str(p.index(i)),fun(i)) for fun in functions for i in p]
#[print(res[0]) for res in results]

CPU times: user 2min 9s, sys: 1.84 s, total: 2min 11s
Wall time: 2min 11s


Acciones y estados para cada problema con diferentes estrateias de búsqueda

In [ ]:
actions = [path_actions(res[1]) for res in results]
states  = [path_states(res[1]) for res in results]

#Printing all inital states
for j in range(len( p)):
  print("Problem "+str(j+1)+":")
  p[j].Print() 

#Printing report for functions
for i in range(len(p)*len(functions)):
  print ('Función: '+str(functions[int(i/len(functions))])[10:-19]+" - Problema "+str(i%3 + 1))
  print ('Estados: ', states[i])
  print ('Numero de estados: ', len(states[i]))
  print ('Acciones: ', actions[i])
  print ('Numero de acciones: ', len(actions[i]), "\n")


Problem 1:
  1 2 7 3 
  5 9 6 4 
  13 11 12 16 
  14 10 8 15 

Problem 2:
  2 7 4 8 
  1 5 3 11 
  14 13 15 10 
  6 9 16 12 

Problem 3:
  1 9 4 8 
  5 6 3 2 
  15 10 11 12 
  13 14 7 16 

Función: breadth_first_search - Problema 1
Estados:  [(1, 2, 7, 3, 5, 9, 6, 4, 13, 11, 12, 16, 14, 10, 8, 15), (1, 2, 7, 3, 5, 9, 6, 4, 13, 11, 8, 12, 14, 10, 15, 16), (1, 2, 7, 3, 5, 6, 8, 4, 13, 9, 11, 12, 14, 10, 15, 16), (1, 2, 3, 4, 5, 6, 7, 8, 13, 9, 11, 12, 14, 10, 15, 16), (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16)]
Numero de estados:  5
Acciones:  [['E+'], ['C-'], ['B-'], ['D-']]
Numero de acciones:  4 

Función: breadth_first_search - Problema 2
Estados:  [(2, 7, 4, 8, 1, 5, 3, 11, 14, 13, 15, 10, 6, 9, 16, 12), (1, 2, 4, 8, 5, 7, 3, 11, 14, 13, 15, 10, 6, 9, 16, 12), (1, 2, 3, 4, 5, 7, 11, 8, 14, 13, 15, 10, 6, 9, 16, 12), (1, 2, 3, 4, 5, 7, 11, 8, 6, 14, 15, 10, 9, 13, 16, 12), (1, 2, 3, 4, 5, 7, 11, 8, 9, 6, 15, 10, 13, 14, 16, 12), (1, 2, 3, 4, 5, 7, 11, 8, 9, 6, 10, 12, 13

Evaluelo para ver cuál es la máxima profundidad que se puede alcanzar en un tiempo razonable con cada estrategia de búsqueda. Reporte los resultados.

Reporte de las funciones BFS, UCS y IDS para 3 problemas diferentes


In [ ]:
%time report(functions, p)

breadth_first_search:
    3,928 nodes |    3,929 goal |    4 cost |     397 actions | rubik2d_problem((1, 2, 7, 3, 5, 9, 6, 4,
   27,135 nodes |   27,136 goal |    6 cost |   2,720 actions | rubik2d_problem((2, 7, 4, 8, 1, 5, 3, 11
  761,740 nodes |  761,741 goal |    8 cost |  76,182 actions | rubik2d_problem((1, 9, 4, 8, 5, 6, 3, 2,
  792,803 nodes |  792,806 goal |   18 cost |  79,299 actions | TOTAL

uniform_cost_search:
    9,940 nodes |      995 goal |    4 cost |     998 actions | rubik2d_problem((1, 2, 7, 3, 5, 9, 6, 4,
  598,580 nodes |   59,859 goal |    6 cost |  59,864 actions | rubik2d_problem((2, 7, 4, 8, 1, 5, 3, 11
7,354,560 nodes |  735,457 goal |    8 cost | 735,464 actions | rubik2d_problem((1, 9, 4, 8, 5, 6, 3, 2,
7,963,080 nodes |  796,311 goal |   18 cost | 796,326 actions | TOTAL

iterative_deepening_search:
    2,400 nodes |    2,381 goal |    4 cost |     244 actions | rubik2d_problem((1, 2, 7, 3, 5, 9, 6, 4,
  102,150 nodes |  102,127 goal |    6 cost |  10,22

## 3. Implemente diferente heurísticas para el problema

Implemente al menos dos heurísticas admisibles y consistentes. Compare A * usando la heurística contra IDS calculando el número de nodos expandidos y el factor de ramificación efectivo, de la misma forma como se hace en la figura 3.29 de [Russell10].

In [ ]:
# p is the vector of states

def astar_manhattan(problem):       return astar_search(problem, h = problem.h)
def astar_null(problem):            return astar_search(problem, h = problem.h0)
def astar_misplaced_tiles(problem): return astar_search(problem, h = problem.h1)
def astar_neighborhood(problem):    return astar_search(problem, h = problem.h2)
def astar_pairs(problem):           return astar_search(problem, h = problem.h3)

Astar = [astar_manhattan, astar_tree_search, astar_pairs, astar_misplaced_tiles, astar_neighborhood, astar_null, iterative_deepening_search]

%time report(Astar, p)

astar_manhattan:
       40 nodes |        5 goal |    4 cost |       8 actions | rubik2d_problem((1, 2, 7, 3, 5, 9, 6, 4,
       60 nodes |        7 goal |    6 cost |      12 actions | rubik2d_problem((2, 7, 4, 8, 1, 5, 3, 11
      980 nodes |       99 goal |   10 cost |     108 actions | rubik2d_problem((1, 9, 4, 8, 5, 6, 3, 2,
    1,080 nodes |      111 goal |   20 cost |     128 actions | TOTAL

astar_tree_search:
       40 nodes |        5 goal |    4 cost |       8 actions | rubik2d_problem((1, 2, 7, 3, 5, 9, 6, 4,
       60 nodes |        7 goal |    6 cost |      12 actions | rubik2d_problem((2, 7, 4, 8, 1, 5, 3, 11
    3,580 nodes |      359 goal |   10 cost |     368 actions | rubik2d_problem((1, 9, 4, 8, 5, 6, 3, 2,
    3,680 nodes |      371 goal |   20 cost |     388 actions | TOTAL

astar_pairs:
      120 nodes |       13 goal |    4 cost |      16 actions | rubik2d_problem((1, 2, 7, 3, 5, 9, 6, 4,
    1,960 nodes |      197 goal |    6 cost |     202 actions | rubik2d_pr

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:69: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


   34,830 nodes |    3,484 goal |    8 cost |   3,491 actions | rubik2d_problem((1, 9, 4, 8, 5, 6, 3, 2,
   36,910 nodes |    3,694 goal |   18 cost |   3,709 actions | TOTAL

astar_misplaced_tiles:
      270 nodes |       28 goal |    4 cost |      31 actions | rubik2d_problem((1, 2, 7, 3, 5, 9, 6, 4,
      130 nodes |       14 goal |    6 cost |      19 actions | rubik2d_problem((2, 7, 4, 8, 1, 5, 3, 11


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:309: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


   74,810 nodes |    7,482 goal |   10 cost |   7,491 actions | rubik2d_problem((1, 9, 4, 8, 5, 6, 3, 2,
   75,210 nodes |    7,524 goal |   20 cost |   7,541 actions | TOTAL

astar_neighborhood:
    1,800 nodes |      181 goal |    4 cost |     184 actions | rubik2d_problem((1, 2, 7, 3, 5, 9, 6, 4,
      140 nodes |       15 goal |    6 cost |      20 actions | rubik2d_problem((2, 7, 4, 8, 1, 5, 3, 11
   83,850 nodes |    8,386 goal |   10 cost |   8,395 actions | rubik2d_problem((1, 9, 4, 8, 5, 6, 3, 2,
   85,790 nodes |    8,582 goal |   20 cost |   8,599 actions | TOTAL

astar_null:
    9,940 nodes |      995 goal |    4 cost |     998 actions | rubik2d_problem((1, 2, 7, 3, 5, 9, 6, 4,
  598,580 nodes |   59,859 goal |    6 cost |  59,864 actions | rubik2d_problem((2, 7, 4, 8, 1, 5, 3, 11
7,354,560 nodes |  735,457 goal |    8 cost | 735,464 actions | rubik2d_problem((1, 9, 4, 8, 5, 6, 3, 2,
7,963,080 nodes |  796,311 goal |   18 cost | 796,326 actions | TOTAL

iterative_deepening_

In [ ]:
#Como era de esperarse la heurística nula lleva a una estrategia UCS, mismos resultados#

In [ ]:
results = [fun(i) for fun in Astar for i in p]
paths   = [path_states(res) for res in results]

d1, d = [len(path) for path in paths], []
for i in range(len(Astar)): d.append(d1[3*i:3*(i+1)]+[sum(d1[3*i:3*(i+1)])])

In [ ]:
def reportN(searchers, problems, verbose=True):
    """Show summary statistics for each searcher (and on each problem unless verbose is false)."""
    for searcher in searchers:
        #print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob   = CountCalls(p)
            soln   = searcher(prob)
            counts = prob._counts; 
            counts.update(actions=len(soln), cost=soln.path_cost)
            total_counts += counts
            #if verbose: report_counts(counts, str(p)[:40])
        return counts['result']


#N1 = reportN(Astar, p)
#[ for a in Astar[:-1]]

N = [[40, 60, 980, 1080], [120, 1960, 34830, 36910], [270, 130, 74810, 75210], 
     [40, 60,3580, 3680], [1800, 140, 83850, 85790], [9940, 598580, 7354560, 7963080], [2400, 102150, 1482140, 14956690]]

def ebf(d, N, possible_bs=[b/100 for b in range(100, 400)]):
    "Effective Branching Factor"
    return min(possible_bs, key=lambda b: abs(N - sum(b**i for i in range(1, d+1))))

EBF = []
for i in range(len(Astar)):
  for j in range(len(p)):
    EBF.append(ebf(d[i][j], N[i][j]))


for i in range(len(Astar)):
  #print("Estrategia "+ Astar[i].__name__)
  for j in range(len(p)):
    pass#print("P"+str(j+1), "ebf ", EBF1[i][j])
  #print("\n")

#print(len(EBF))

Ebf = [[], [], []]
for j in range(0,len(EBF),3):
    Ebf[0].append(str(EBF[j]))
    Ebf[1].append(str(EBF[j+1]))
    Ebf[2].append(str(EBF[j+2]))

#print(EBF, len(EBF))
#print(Ebf)
names = [str(search.__name__) for search in Astar]

#print(names[0])
print ("   "+'   '.join(names))
print ("p1  "+'                  '.join(Ebf[0]))
print ("p2  "+'                  '.join(Ebf[1]))
print ("p3  "+'                  '.join(Ebf[2]))
#[print(name) for name in names]
#print("P1", Ebf[0])
#print("P2", Ebf[1])
#print("P3", Ebf[2])

   astar_manhattan   astar_tree_search   astar_pairs   astar_misplaced_tiles   astar_neighborhood   astar_null   iterative_deepening_search
p1  1.8                  2.34                  2.81                  1.8                  3.99                  3.99                  3.99
p2  1.56                  2.77                  1.79                  1.56                  1.81                  3.99                  3.99
p3  1.73                  2.47                  3.35                  1.97                  2.69                  3.99                  3.99


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


In [ ]:
print(N1)